In [2]:
import re, string, unicodedata
import nltk
import contractions
#from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
#from contractions import CONTRACTION_MAP

In [4]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    #text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text



# Define function to expand contractions
def expand_contractions(text):
    contractions_pattern = re.compile('({})'.format('|'.join(CONTRACTION_MAP.keys())),flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = CONTRACTION_MAP.get(match)\
                        if CONTRACTION_MAP.get(match)\
                        else CONTRACTION_MAP.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
    
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text


# special_characters removal
def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text


def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words


def remove_punctuation_and_splchars(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_word = remove_special_characters(new_word, True)
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

#stopword_list= stopwords.words('english')
#stopword_list.remove('no')
#stopword_list.remove('not')
#stopword_list.remove('headphone')
#stopword_list.remove('headphones')
#stopword_list.remove('earbuds')
#stopword_list.remove('bud')
#stopword_list.remove('ear')
#stopword_list.remove('sony')
#stopword_list.remove('product')


def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopword_list:
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation_and_splchars(words)
    words = remove_stopwords(words)
    return words

def lemmatize(words):
    lemmas = lemmatize_verbs(words)
    return lemmas

In [5]:
def normalize_and_lemmaize(input):
    sample = denoise_text(input)
    #sample = expand_contractions(sample)
    sample = remove_special_characters(sample)
    words = nltk.word_tokenize(sample)
    words = normalize(words)
    lemmas = lemmatize(words)
    return ' '.join(lemmas)

In [8]:
import pandas as pd

RECIPES_PATH = "./data/saree_data.csv"
recipe_df = pd.read_csv(RECIPES_PATH)
recipe_df.head(5)

,ingredients,recipe_urls,recipe_name
0,(A Govt. of West Bengal Enterprises) Handloom ...,0.jpeg,saree tantuja bengal handloom
1,Women's Silk Saree With Blouse Piece,1.jpeg,saree trilok fab
2,Women's Georgette Net Embroidered Saree with B...,2.jpeg,saree nine sister
3,Women's Crepe Saree with Blouse (GLE$CHICKOO C...,3.jpeg,saree gauri laxmi enterprise
4,Women's Cotton Blend Digital Butta Printed Sar...,4.jpeg,saree sourbh


In [10]:
recipe_df["desc"] = recipe_df['ingredients']
# change the way the sentence is arranged in the data
recipe_df['ingredients'] = recipe_df['ingredients'].map(str) + ',' + recipe_df['recipe_name'].map(str)
recipe_df['ingredients'] = recipe_df['ingredients'].str.split()

#recipe_df['ingredients_parsed'] = recipe_df['ingredients'].apply(lambda x: normalize_and_lemmaize(x))
    
recipe_df['ingredients']

0      [['(A',, 'Govt.',, 'of',, 'West',, 'Bengal',, ...
1      [["Women's",, 'Silk',, 'Saree',, 'With',, 'Blo...
2      [["Women's",, 'Georgette',, 'Net',, 'Embroider...
3      [["Women's",, 'Crepe',, 'Saree',, 'with',, 'Bl...
4      [["Women's",, 'Cotton',, 'Blend',, 'Digital',,...
                             ...                        
995    [['GOGURL',, "Women's",, 'Lycra',, 'Cotton',, ...
996    [["Women's",, 'Multicolour',, 'Sweet',, 'Heart...
997    [['Round',, 'Neck',, 'Women',, 'Blouse,blouse'...
998    [["Women's",, 'Navy',, 'Blue',, 'Phantom',, 'B...
999    [['Black',, 'Pure',, 'Cotton',, 'Handloom',, '...
Name: ingredients, Length: 1000, dtype: object

In [27]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/sangames/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sangames/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sangames/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [31]:

# calculate raw tokens in order to measure of cleaned tokens

from nltk.tokenize import word_tokenize
raw_tokens=len([w for t in (recipe_df['desc'].apply(word_tokenize)) for w in t])
print('Number of raw tokens: {}'.format(raw_tokens))

TypeError: expected string or bytes-like object

In [28]:
recipe_df['ingredients_parsed'] = recipe_df['recipe_name'].apply(lambda x: normalize_and_lemmaize(x))

NameError: name 'stopword_list' is not defined

In [21]:
recipe_df['desc']

0      [(A, Govt., of, West, Bengal, Enterprises), Ha...
1      [Women's, Silk, Saree, With, Blouse, Piece,sar...
2      [Women's, Georgette, Net, Embroidered, Saree, ...
3      [Women's, Crepe, Saree, with, Blouse, (GLE$CHI...
4      [Women's, Cotton, Blend, Digital, Butta, Print...
                             ...                        
995    [GOGURL, Women's, Lycra, Cotton, Stretchable, ...
996    [Women's, Multicolour, Sweet, Heart, Neckline,...
997       [Round, Neck, Women, Blouse,blouse, mooldhani]
998    [Women's, Navy, Blue, Phantom, Blouse, With, R...
999    [Black, Pure, Cotton, Handloom, Back-Open, Blo...
Name: desc, Length: 1000, dtype: object